# Coursera Capstone Project

This is fulfilment of coursera course and will be mainly used for capstone project

In [64]:
#scientific library
import pandas as pd
import numpy as np

In [399]:
print("Hello Capstone Project Course!!")

Hello Capstone Project Course!


In [66]:
#web scraping library
import requests
from bs4 import BeautifulSoup
import time

In [798]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [799]:
#create a get request to a URL

r = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

## Data Import and Sanitation

In [800]:
#parse html request using BeutifulSOup
df = []
for b in r:
    html = requests.get(r).text
    soup = BeautifulSoup(html, "html.parser")
    #identify table we want to scrape
    table = soup.find('table', {"class" : "wikitable"})
    #try clause to skip any companies with missing/empty board member tables
    try:
        #loop through table, grab each of the 4 columns shown (try one of the links yourself to see the layout)
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            #print(len(cols))
            if len(cols) == 3:
                df.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))
    except: pass  

In [801]:
#to array
df = np.asarray(df)
#convert new array to dataframe
df = pd.DataFrame(df)

In [802]:
df.columns = ['Postcode', 'Borough', 'Neighborhood']
df.count() #18207 all records
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Check more than one neighborhood can exist in one postal code area.

In [803]:
na_borough = df[df.Borough == "Not assigned"]
na_borough.count() # 4851

Postcode        4851
Borough         4851
Neighborhood    4851
dtype: int64

**NOTE:** Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [804]:
#remove row that has Borough value 'Not Assigned'
df = df[df.Borough != "Not assigned"]

In [805]:
df.count() # 18207 - 4851 = 13356

Postcode        13356
Borough         13356
Neighborhood    13356
dtype: int64

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [806]:
#let's check how many not assigned neighborhood
t = df.loc[(df['Neighborhood'] == "Not assigned") & (df['Borough'] != "Not assigned")] 
t.count() #63
t.head() #all are under Borough == Queen's Park and has Postcode value of M7A

,Postcode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned
297,M7A,Queen's Park,Not assigned
586,M7A,Queen's Park,Not assigned
875,M7A,Queen's Park,Not assigned
1164,M7A,Queen's Park,Not assigned


Since I know that all not assighned neighborhood is under Qeen's Park

In [807]:
df['Neighborhood'].loc[(df['Neighborhood'] == "Not assigned") & (df['Borough'] != "Not assigned")]= "Queen's Park"

In [808]:
t = df[df['Borough'] == "Queen's Park"]
t.count()

Postcode        63
Borough         63
Neighborhood    63
dtype: int64

Check values of Borough and Neighborhood

In [809]:
df[df.Borough == "Queen's Park"].head()

,Postcode,Borough,Neighborhood
8,M7A,Queen's Park,Queen's Park
297,M7A,Queen's Park,Queen's Park
586,M7A,Queen's Park,Queen's Park
875,M7A,Queen's Park,Queen's Park
1164,M7A,Queen's Park,Queen's Park


More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [853]:
#combine rows having similar postal code area
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join)
df = pd.DataFrame(df)

In [855]:
df.head()

,,Neighborhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern,Rouge,Malvern,Roug..."
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ..."
M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn..."
M1G,Scarborough,"Woburn,Woburn,Woburn,Woburn,Woburn,Woburn,Wobu..."
M1H,Scarborough,"Cedarbrae,Cedarbrae,Cedarbrae,Cedarbrae,Cedarb..."


In [856]:
df.head() #the index neighborhood is flying

,,Neighborhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern,Rouge,Malvern,Roug..."
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ..."
M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn..."
M1G,Scarborough,"Woburn,Woburn,Woburn,Woburn,Woburn,Woburn,Wobu..."
M1H,Scarborough,"Cedarbrae,Cedarbrae,Cedarbrae,Cedarbrae,Cedarb..."


In [857]:
df = df.reset_index()

In [858]:
df.tail() #index neighborhood went back

,Postcode,Borough,Neighborhood
98,M9N,York,"Weston,Weston,Weston,Weston,Weston,Weston,West..."
99,M9P,Etobicoke,"Westmount,Westmount,Westmount,Westmount,Westmo..."
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
102,M9W,Etobicoke,"Northwest,Northwest,Northwest,Northwest,Northw..."


In [814]:
df.Postcode.unique()
df.count() #we have 103 unique values 

Postcode        103
Borough         103
Neighborhood    103
dtype: int64

In [861]:
#delete repeating value in a cell
df['Neighborhood']=df['Neighborhood'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")


In [862]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"HighlandCreek,PortUnion,RougeHill"
2,M1E,Scarborough,"Guildwood,WestHill,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [863]:
df.shape

(103, 3)

## Combine Latitude and Longitude in our existing data frame

In [864]:
#import coordinates
coor_list = pd.read_csv('http://cocl.us/Geospatial_data')

In [865]:
coor_list = coor_list.rename(columns={'Postal Code': 'Postcode'})


In [870]:
coor_list.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [871]:
#merge df and coor_list
post_coor = pd.merge(df, coor_list, on="Postcode", how="left")

In [873]:
#check if successful
post_coor.count() #103
post_coor.dtypes # make sure that coordinates are in float format

Postcode         object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

## Visualisation using Map

#### Use geopy library to get the latitude and longitude values of Toronto Canada

In [874]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [875]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat,lng,bourough, neighborhood in zip(post_coor['Latitude'],post_coor['Longitude'],post_coor['Borough'],post_coor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, bourough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)   
    
    
map_toronto

### Define Foursquare Credentials and Version

In [876]:
CLIENT_ID = 'VS31FBLWDPVVEFZJILDIOQ4L4YOV5PUKNMWAAP23DSKTNRG0' # your Foursquare ID
CLIENT_SECRET = '0PF4BMKGQWD01UTSQ5XAKQX1EW3VDXFQUT2D4GQQB1QVAXHG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VS31FBLWDPVVEFZJILDIOQ4L4YOV5PUKNMWAAP23DSKTNRG0
CLIENT_SECRET:0PF4BMKGQWD01UTSQ5XAKQX1EW3VDXFQUT2D4GQQB1QVAXHG


In [877]:
post_coor.loc[0, 'Neighborhood']

'Rouge,Malvern'

Get the neighborhood's latitude and longitude values.

In [879]:
neighborhood_latitude = post_coor.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = post_coor.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = post_coor.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [883]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=VS31FBLWDPVVEFZJILDIOQ4L4YOV5PUKNMWAAP23DSKTNRG0&client_secret=0PF4BMKGQWD01UTSQ5XAKQX1EW3VDXFQUT2D4GQQB1QVAXHG&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

Send the GET request and examine the resutls

In [885]:
results = requests.get(url).json()
#results

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [886]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [887]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


### Neighborhoods Exploration in Toronto

In [888]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
   # return(nearby_venues)

In [890]:
toronto_venues = getNearbyVenues(names=post_coor['Neighborhood'],
                                   latitudes=post_coor['Latitude'],
                                   longitudes=post_coor['Longitude']
                                  )

Rouge,Malvern
HighlandCreek,PortUnion,RougeHill
Guildwood,WestHill,Morningside
Woburn
Cedarbrae
ScarboroughVillage
EastBirchmountPark,KennedyPark,Ionview
GoldenMile,Oakridge,Clairlea
Cliffcrest,ScarboroughVillageWest,Cliffside
CliffsideWest,BirchCliff
DorsetPark,WexfordHeights,ScarboroughTownCentre
Maryvale,Wexford
Agincourt
ClarksCorners,Sullivan,TamO'Shanter
SteelesEast,L'AmoreauxEast,AgincourtNorth,Milliken
SteelesWest,L'AmoreauxWest
UpperRouge
HillcrestVillage
Oriole,Fairview,HenryFarm
BayviewVillage
YorkMills,SilverHills
Newtonbrook,Willowdale
WillowdaleSouth
YorkMillsWest
WillowdaleWest
Parkwoods
DonMillsNorth
DonMillsSouth,FlemingdonPark
DownsviewNorth,BathurstManor,WilsonHeights
NorthwoodPark,YorkUniversity
CFBToronto,DownsviewEast
DownsviewWest
DownsviewCentral
DownsviewNorthwest
VictoriaVillage
ParkviewHill,WoodbineGardens
WoodbineHeights
TheBeaches
Leaside
ThorncliffePark
EastToronto
TheDanforthWest,Riverdale
IndiaBazaar,TheBeachesWest
StudioDistrict
LawrencePark
DavisvilleN

Let's check the size of the resulting dataframe

In [891]:
print(toronto_venues.shape)
toronto_venues.head()

(2219, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"HighlandCreek,PortUnion,RougeHill",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,WestHill,Morningside",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,WestHill,Morningside",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Let's check how many venues were returned for each neighborhood

In [892]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
BayviewVillage,4,4,4,4,4,4
BerczyPark,57,57,57,57,57,57
"Brockton,ParkdaleVillage,ExhibitionPlace",20,20,20,20,20,20
BusinessReplyMailProcessingCentre969Eastern,16,16,16,16,16,16
"CFBToronto,DownsviewEast",3,3,3,3,3,3
Caledonia-Fairbanks,6,6,6,6,6,6
CanadaPostGatewayProcessingCentre,10,10,10,10,10,10
Cedarbrae,7,7,7,7,7,7


Let's find out how many unique categories can be curated from all the returned venues

In [893]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


### Analyze Each Neighborhood

In [894]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size

In [895]:
toronto_onehot.shape

(2219, 272)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶

In [896]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

Let's confirm the new size

In [897]:
toronto_grouped.shape

(99, 272)

Let's print each neighborhood along with the top 5 most common venues

In [898]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0    Skating Rink  0.25
1          Lounge  0.25
2  Breakfast Spot  0.25
3  Sandwich Place  0.25
4  Medical Center  0.00


----BayviewVillage----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4        Movie Theater  0.00


----BerczyPark----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2         Cheese Shop  0.04
3      Farmers Market  0.04
4  Seafood Restaurant  0.04


----Brockton,ParkdaleVillage,ExhibitionPlace----
            venue  freq
0     Coffee Shop  0.10
1  Breakfast Spot  0.10
2            Café  0.10
3       Nightclub  0.05
4             Bar  0.05


----BusinessReplyMailProcessingCentre969Eastern----
                venue  freq
0       Burrito Place  0.06
1       Auto Workshop  0.06
2       Moving Target  0.06
3  Light Rail Station  0.06
4             Brewery  0.06


----CFBToronto,DownsviewEast---

                      venue  freq
0                      Pool  0.25
1  Mediterranean Restaurant  0.25
2                   Dog Run  0.25
3               Golf Course  0.25
4               Yoga Studio  0.00


----HumberBay,KingswayParkSouthEast,MimicoNE,King'sMillPark,OldMillSouth,TheQueenswayEast,Sunnylea,RoyalYorkSouthEast----
                        venue  freq
0                   Locksmith  0.33
1  Construction & Landscaping  0.33
2              Baseball Field  0.33
3                 Yoga Studio  0.00
4   Middle Eastern Restaurant  0.00


----HumberSummit----
                             venue  freq
0                      Pizza Place   0.5
1              Empanada Restaurant   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----Humewood-Cedarvale----
                             venue  freq
0                            Field  0.25
1                            Trail  0.25
2                       Playground 

                 venue  freq
0                 Park   0.5
1    Mobile Phone Shop   0.5
2          Yoga Studio   0.0
3   Mexican Restaurant   0.0
4  Monument / Landmark   0.0


----SteelesEast,L'AmoreauxEast,AgincourtNorth,Milliken----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0


----SteelesWest,L'AmoreauxWest----
                  venue  freq
0  Fast Food Restaurant  0.17
1    Chinese Restaurant  0.17
2           Pizza Place  0.08
3           Coffee Shop  0.08
4        Breakfast Spot  0.08


----StnAPOBoxes25TheEsplanade----
                  venue  freq
0           Coffee Shop  0.11
1                  Café  0.04
2            Restaurant  0.04
3  Fast Food Restaurant  0.03
4    Italian Restaurant  0.03


----StudioDistrict----
                venue  freq
0                Café  0.10
1    

Let's put that into a pandas dataframe. First, let's write a function to sort the venues in descending order.

In [900]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [901]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Sandwich Place,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Dumpling Restaurant
1,BayviewVillage,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,BerczyPark,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Seafood Restaurant,Farmers Market,Steakhouse,Italian Restaurant
3,"Brockton,ParkdaleVillage,ExhibitionPlace",Coffee Shop,Café,Breakfast Spot,Pet Store,Bar,Furniture / Home Store,Italian Restaurant,Convenience Store,Nightclub,Performing Arts Venue
4,BusinessReplyMailProcessingCentre969Eastern,Garden,Burrito Place,Fast Food Restaurant,Farmers Market,Spa,Moving Target,Skate Park,Garden Center,Light Rail Station,Pizza Place


##  Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [902]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [913]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = post_coor

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Department Store
1,M1C,Scarborough,"HighlandCreek,PortUnion,RougeHill",43.784535,-79.160497,0.0,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
2,M1E,Scarborough,"Guildwood,WestHill,Morningside",43.763573,-79.188711,0.0,Electronics Store,Spa,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Women's Store,Dog Run,Dim Sum Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store


**NOTE:** There are NaN values upon merging, to aid this issue, all NaN rows will be remove

In [914]:
toronto_merged = toronto_merged.dropna()

In [915]:
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Department Store
1,M1C,Scarborough,"HighlandCreek,PortUnion,RougeHill",43.784535,-79.160497,0.0,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
2,M1E,Scarborough,"Guildwood,WestHill,Morningside",43.763573,-79.188711,0.0,Electronics Store,Spa,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Women's Store,Dog Run,Dim Sum Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Pharmacy,Korean Restaurant,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store


In [916]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

In [917]:
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Department Store
1,M1C,Scarborough,"HighlandCreek,PortUnion,RougeHill",43.784535,-79.160497,0,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
2,M1E,Scarborough,"Guildwood,WestHill,Morningside",43.763573,-79.188711,0,Electronics Store,Spa,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Women's Store,Dog Run,Dim Sum Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Pharmacy,Korean Restaurant,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store


Finally, let's visualize the resulting clusters

In [918]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
         [lat, lon],
         radius=5,
         popup=label,
         color=rainbow[cluster-1],
         fill=True,
         fill_color=rainbow[cluster-1],
         fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


## Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster. 

#### CLUSTER 1

In [919]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Print Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Department Store
1,Scarborough,0,Bar,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Farmers Market
2,Scarborough,0,Electronics Store,Spa,Pizza Place,Rental Car Location,Breakfast Spot,Mexican Restaurant,Medical Center,Women's Store,Dog Run,Dim Sum Restaurant
3,Scarborough,0,Coffee Shop,Pharmacy,Korean Restaurant,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Drugstore
4,Scarborough,0,Fried Chicken Joint,Hakka Restaurant,Athletics & Sports,Bakery,Bank,Thai Restaurant,Caribbean Restaurant,Doner Restaurant,Diner,Discount Store
6,Scarborough,0,Department Store,Bus Station,Coffee Shop,Discount Store,Convenience Store,Drugstore,Diner,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,0,Bus Line,Bakery,Park,Intersection,Fast Food Restaurant,Metro Station,Soccer Field,Construction & Landscaping,Convenience Store,Ethiopian Restaurant
8,Scarborough,0,Motel,American Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Department Store
9,Scarborough,0,Café,College Stadium,Skating Rink,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,Scarborough,0,Indian Restaurant,Vietnamese Restaurant,Furniture / Home Store,Thrift / Vintage Store,Chinese Restaurant,Pet Store,Latin American Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore


#### CLUSTER 2

In [920]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1,Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
25,North York,1,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
30,North York,1,Park,Bus Stop,Airport,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
40,East York,1,Park,Convenience Store,Metro Station,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Women's Store
50,Downtown Toronto,1,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
64,Central Toronto,1,Park,Trail,Sushi Restaurant,Jewelry Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Department Store
74,York,1,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Airport Food Court,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant
79,North York,1,Basketball Court,Park,Bakery,Construction & Landscaping,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
90,Etobicoke,1,Park,River,Pool,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
98,York,1,Convenience Store,Park,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store


#### CLUSTER 3

In [921]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
48,Central Toronto,2,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


#### CLUSTER 4

In [922]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,3,Park,Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
94,Etobicoke,3,Bank,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,Farmers Market


#### CLUSTER 5

In [923]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,4,Locksmith,Baseball Field,Construction & Landscaping,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
97,North York,4,Baseball Field,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store,Farmers Market
